# Polygon Image Extraction
Notebook version of extract_polygon_images.py with staged cells for installs, helpers, and execution.

In [ ]:
# Install required libraries (uncomment if running in a fresh Colab/runtime)
# !pip install geopandas rasterio shapely pyproj matplotlib numpy

In [ ]:
import argparse
import json
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.windows import Window
from shapely.geometry import MultiPolygon
from pyproj import Geod

# Default input files
DEFAULT_LAYERS_FILE = Path("./data/ML_labeling_UNITA.gpkg")
DEFAULT_ORTHO_TIF = Path("./data/CerroUnita_ortomosaico.tif")
DEFAULT_OUTPUT_DIR = Path("extracted_geoglifs")

## Bounding Box Calculations

In [ ]:
def calculate_bbox_size_meters(bounds, crs):
    """Calculate width, height, and area in meters for a bounding box."""
    from shapely.geometry import Point
    from shapely.ops import transform
    from pyproj import Transformer

    minx, miny, maxx, maxy = bounds

    if crs and crs.to_epsg() != 4326:
        transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
        p1_wgs84 = transform(transformer.transform, Point(minx, miny))
        p2_wgs84 = transform(transformer.transform, Point(maxx, miny))
        p3_wgs84 = transform(transformer.transform, Point(minx, maxy))
        minx_wgs, miny_wgs = p1_wgs84.x, p1_wgs84.y
        maxx_wgs, miny_wgs2 = p2_wgs84.x, p2_wgs84.y
        minx_wgs2, maxy_wgs = p3_wgs84.x, p3_wgs84.y
    else:
        minx_wgs, miny_wgs, maxx_wgs, maxy_wgs = minx, miny, maxx, maxy

    geod = Geod(ellps="WGS84")
    _, _, width_m = geod.inv(minx_wgs, miny_wgs, maxx_wgs, miny_wgs)
    _, _, height_m = geod.inv(minx_wgs, miny_wgs, minx_wgs, maxy_wgs)

    return {
        "width_m": abs(width_m),
        "height_m": abs(height_m),
        "area_m2": abs(width_m * height_m),
    }

## Save Helpers

In [ ]:
def save_image_no_axes(array, output_path: Path, transform=None, crs=None, is_rgb=False):
    """Save image without axes as TIF."""
    if len(array.shape) == 2:
        array = array[np.newaxis, ...]

    count, height, width = array.shape

    with rasterio.open(
        str(output_path),
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=count,
        dtype=array.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(array)


def save_jpeg_no_axes(array, output_path: Path, dpi=150):
    """Save array as JPEG without axes or borders."""
    if array.dtype != np.uint8:
        arr_min, arr_max = array.min(), array.max()
        array_norm = (array - arr_min) / (arr_max - arr_min) if arr_max > arr_min else array
        array = (array_norm * 255).astype(np.uint8)

    h, w = array.shape[:2]
    fig_width = w / dpi
    fig_height = h / dpi

    fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
    ax = fig.add_axes((0, 0, 1, 1))
    ax.axis("off")

    if len(array.shape) == 3:
        ax.imshow(array)
    else:
        ax.imshow(array, cmap="gray")

    plt.savefig(str(output_path), dpi=dpi, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


def save_overlay_jpeg(array, polygons, transform, output_path: Path, dpi=150):
    """Save array with polygon overlay as JPEG without axes."""
    if len(array.shape) == 3 and array.shape[0] in [3, 4]:
        array = array.transpose(1, 2, 0)

    if array.dtype != np.uint8:
        arr_min, arr_max = array.min(), array.max()
        array_norm = (array - arr_min) / (arr_max - arr_min) if arr_max > arr_min else array
        array = (array_norm * 255).astype(np.uint8)

    if array.shape[2] > 3:
        array = array[:, :, :3]

    h, w = array.shape[:2]
    fig_width = w / dpi
    fig_height = h / dpi

    fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
    ax = fig.add_axes((0, 0, 1, 1))
    ax.axis("off")

    x0, y0 = transform * (0, 0)
    x1, y1 = transform * (w, h)
    extent = (x0, x1, y1, y0)

    ax.imshow(array, extent=extent, interpolation="nearest")

    for poly in polygons:
        if hasattr(poly, "exterior"):
            x, y = poly.exterior.xy
            ax.plot(x, y, color="yellow", linewidth=2)

    ax.set_xlim(extent[0], extent[1])
    ax.set_ylim(extent[2], extent[3])

    plt.savefig(str(output_path), dpi=dpi, bbox_inches="tight", pad_inches=0)
    plt.close(fig)

## Extraction Logic

In [ ]:
def extract_polygon_images(polygon_idx, geometry, gdf_crs, polygon_class, ortho_path: Path, output_dir: Path):
    """Extract ortho crops, overlay images, and metadata for a single polygon."""
    minx, miny, maxx, maxy = geometry.bounds

    with rasterio.open(str(ortho_path)) as ortho:
        row_min, col_min = ortho.index(minx, maxy)
        row_max, col_max = ortho.index(maxx, miny)

        ortho_win = Window.from_slices((row_min, row_max), (col_min, col_max))
        ortho_chunk = ortho.read(window=ortho_win)
        ortho_transform = ortho.window_transform(ortho_win)
        ortho_crs = ortho.crs

        width = maxx - minx
        height = maxy - miny
        padding_x = width * 0.05
        padding_y = height * 0.05

        minx_padded = minx - padding_x
        maxx_padded = maxx + padding_x
        miny_padded = miny - padding_y
        maxy_padded = maxy + padding_y

        row_min_padded, col_min_padded = ortho.index(minx_padded, maxy_padded)
        row_max_padded, col_max_padded = ortho.index(maxx_padded, miny_padded)

        overlay_win = Window.from_slices((row_min_padded, row_max_padded), (col_min_padded, col_max_padded))
        overlay_chunk = ortho.read(window=overlay_win)
        overlay_transform = ortho.window_transform(overlay_win)

    polygons = [p for p in geometry.geoms] if isinstance(geometry, MultiPolygon) else [geometry]
    bbox_size = calculate_bbox_size_meters(geometry.bounds, gdf_crs)

    base_name = f"geoglif_{polygon_idx:04d}"
    tif_path = output_dir / f"{base_name}_ortho.tif"
    jpeg_path = output_dir / f"{base_name}_ortho.jpg"
    overlay_path = output_dir / f"{base_name}_overlay.jpg"
    metadata_path = output_dir / f"{base_name}_metadata.json"

    save_image_no_axes(ortho_chunk, tif_path, ortho_transform, ortho_crs, is_rgb=True)
    ortho_rgb = ortho_chunk[:3].transpose(1, 2, 0)
    save_jpeg_no_axes(ortho_rgb, jpeg_path)
    save_overlay_jpeg(overlay_chunk, polygons, overlay_transform, overlay_path)

    metadata = {
        "polygon_index": polygon_idx,
        "class": int(polygon_class),
        "bounds": {"minx": minx, "miny": miny, "maxx": maxx, "maxy": maxy},
        "bbox_size_meters": bbox_size,
        "crs": str(gdf_crs),
        "image_shape": {
            "height": ortho_chunk.shape[1],
            "width": ortho_chunk.shape[2],
            "channels": ortho_chunk.shape[0],
        },
        "files": {
            "ortho_tif": tif_path.name,
            "ortho_jpeg": jpeg_path.name,
            "overlay_jpeg": overlay_path.name,
        },
    }

    metadata_path.write_text(json.dumps(metadata, indent=2))
    return metadata

## CLI / Batch Helpers

In [ ]:
def parse_args():
    """Parse command line arguments."""
    parser = argparse.ArgumentParser(
        description="Extract images from geo-referenced data for each polygon in the geopackage.",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  # Process all polygons
  python extract_polygon_images.py

  # Process only the first 10 polygons
  python extract_polygon_images.py --limit 10

  # Process first 20 polygons with custom output directory
  python extract_polygon_images.py --limit 20 --output custom_output

  # Use custom input files
  python extract_polygon_images.py --layers my_layers.gpkg --ortho my_ortho.tif
        """
    )

    parser.add_argument(
        "--layers",
        type=Path,
        default=DEFAULT_LAYERS_FILE,
        help=f"Path to the geopackage file containing polygons (default: {DEFAULT_LAYERS_FILE})",
    )

    parser.add_argument(
        "--ortho",
        type=Path,
        default=DEFAULT_ORTHO_TIF,
        help=f"Path to the orthomosaic TIF file (default: {DEFAULT_ORTHO_TIF})",
    )

    parser.add_argument(
        "--output",
        type=Path,
        default=DEFAULT_OUTPUT_DIR,
        help=f"Output directory for extracted images (default: {DEFAULT_OUTPUT_DIR})",
    )

    parser.add_argument(
        "--limit",
        type=int,
        default=None,
        help="Limit processing to the first N polygons (default: process all)",
    )

    return parser.parse_args()

def main():
    args = parse_args()
    args.output.mkdir(parents=True, exist_ok=True)

    print(f"Loading polygons from {args.layers}...")
    gdf = gpd.read_file(str(args.layers))
    print(f"Loaded {len(gdf)} polygons")
    print(f"Original CRS: {gdf.crs}")

    if args.limit is not None:
        print(f"Limiting to first {args.limit} polygons")
        gdf = gdf.head(args.limit)

    with rasterio.open(str(args.ortho)) as ortho:
        ortho_crs = ortho.crs
        print(f"Ortho CRS: {ortho_crs}")

    if gdf.crs != ortho_crs:
        print(f"Converting from {gdf.crs} to {ortho_crs}")
        gdf = gdf.to_crs(ortho_crs)

    all_metadata = []
    for idx, row in gdf.iterrows():
        polygon_class = row["class"]
        class_name = {1: "geo", 2: "ground", 3: "road"}.get(polygon_class, "unknown")
        fid = idx
        print(
            f"\nProcessing polygon {idx + 1}/{len(gdf)} (FID: {fid}, class: {polygon_class} - {class_name})..."
        )
        try:
            metadata = extract_polygon_images(
                idx, row.geometry, gdf.crs, polygon_class, args.ortho, args.output
            )
            all_metadata.append(metadata)
            print(
                f"  Bounding box: {metadata['bbox_size_meters']['width_m']:.2f}m x "
                f"{metadata['bbox_size_meters']['height_m']:.2f}m "
                f"(area: {metadata['bbox_size_meters']['area_m2']:.2f} m²)"
            )
        except Exception as exc:
            print(f"  Error processing polygon {idx}: {exc}")

    summary_path = args.output / "summary.json"
    summary_data = {
        "total_polygons": len(gdf),
        "processed_polygons": len(all_metadata),
        "output_directory": str(args.output),
        "source_files": {"layers": str(args.layers), "ortho": str(args.ortho)},
        "polygons": all_metadata,
    }
    summary_path.write_text(json.dumps(summary_data, indent=2))

    print(f"\n\nDone! Processed {len(all_metadata)} polygons.")
    print(f"Output directory: {args.output}")
    print(f"Summary saved to: {summary_path}")

## Run (Notebook-Friendly)

In [ ]:
# Adjust paths/limit as needed for notebook runs
layers_path = DEFAULT_LAYERS_FILE
ortho_path = DEFAULT_ORTHO_TIF
output_dir = DEFAULT_OUTPUT_DIR
limit = None  # set an int to limit polygons

# Uncomment to run directly in the notebook
# args = argparse.Namespace(layers=layers_path, ortho=ortho_path, output=output_dir, limit=limit)
# args.output.mkdir(parents=True, exist_ok=True)
# gdf = gpd.read_file(str(args.layers))
# if args.limit is not None:
#     gdf = gdf.head(args.limit)
# with rasterio.open(str(args.ortho)) as ortho_src:
#     ortho_crs = ortho_src.crs
# if gdf.crs != ortho_crs:
#     gdf = gdf.to_crs(ortho_crs)
# all_metadata = []
# for idx, row in gdf.iterrows():
#     metadata = extract_polygon_images(idx, row.geometry, gdf.crs, row['class'], args.ortho, args.output)
#     all_metadata.append(metadata)
# summary = {
#     "total_polygons": len(gdf),
#     "processed_polygons": len(all_metadata),
#     "output_directory": str(args.output),
#     "source_files": {"layers": str(args.layers), "ortho": str(args.ortho)},
#     "polygons": all_metadata,
# }
# (args.output / "summary.json").write_text(json.dumps(summary, indent=2))
# print(f"Done! Processed {len(all_metadata)} polygons. Summary saved to {args.output / 'summary.json'}")

# Or call main() to reuse CLI defaults (parses sys.argv)
# main()